In [1]:
import requests
import time

# 질문 리스트
questions = {
    "interest" : "어떤 분야에 관심이 많고, 주로 즐기는 취미는 무엇인가요? (예: 요리, 여행, 게임, 뷰티 등)",
    "contents" : "영상 콘텐츠를 제작할 때 선호하는 유형이 있나요? (예: 브이로그, 리뷰, 튜토리얼, 예능 등)",
    "target" : "주 타겟으로 삼고 싶은 연령대나 성별, 지역 등이 있나요?",
    "time" : "일주일에 영상 제작에 할애할 수 있는 시간이 어느정도 인가요?",
    "budget" : "현재 보유하고 있는 장비와 앞으로의 예산 계획은 어떻게 되나요?",
    "creativity" : "평소에 떠올렸던 재미있거나 유익한 콘텐츠 아이디어가 있다면 공유해주세요.",
    "goal" : "유튜버로서 이루고 싶은 장기적인 목표나 꿈이 있나요?"
}

class Chatbot:
    def __init__(self, user_states):
        # 사용자 상태 관리 딕셔너리
        self.user_states = user_states
        self.state_to_ques = {0:"interest", 1:"contents", 2:"target", 3:"time", 4:"budget", 5:"creativity", 6:"goal"}

    # 다음 질문 가져오기
    def get_next_question(self):        
        if self.user_states["current_index"] < len(questions):
            question = questions[self.state_to_ques[self.user_states["current_index"]]]
            return question, self.state_to_ques[self.user_states["current_index"]]
        else:
            return None, -1

    # 사용자 답변 처리
    def handle_user_answer(self, question, answer):
        self.user_states["answers"][question] = answer
        self.user_states["current_index"] += 1

        # 다음 질문 가져오기
        next_question, question_id = self.get_next_question()
        if next_question != None:
            return next_question, question_id
        else:
            return None, -1
        

class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id
        self._max_retries = 5

    def execute(self, completion_request):
        print('답변을 생성하고 있습니다...')
        headers = {
            "Authorization": self._api_key,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id,
            "Content-Type": "application/json; charset=utf-8",
        }

        retries = 0  # 현재 재시도 횟수
        while retries < self._max_retries:
            # POST 요청 보내기
            response = requests.post(
                self._host + "/testapp/v1/chat-completions/HCX-003",
                headers=headers,
                json=completion_request
            )

            # 응답 상태 확인
            if response.status_code == 200:
                response_data = response.json()
                return response_data['result']["message"]["content"]
            elif response.status_code == 429:  # Too Many Requests
                print(f"Rate limit exceeded. Retrying after {1} seconds...")
                time.sleep(10)
                print(response.json())
                retries+=1
    

In [ ]:
# HyperCLOVA 호출 함수
import requests

HYPERCLOVA_API_URL = "https://clovastudio.stream.ntruss.com"
HYPERCLOVA_API_KEY = "Bearer nv-f5786fde571f424786ed0823986ca992h3P1"


def call_hyperclova(answers):
    prompt = [{
        "role": "system",
        "content": """이제 막 유튜버의 꿈을 펼치려고 하는 사람이 있습니다. 이 초보 유튜버는 어떤 영상을 만들어야 할지, 채널의 방향을 어떻게 잡아야 할지 고민하고 있습니다. 초보 유튜버의 성공이 당신의 분석에 달렸습니다. 이 사람에게 맞는 채널 방향성과 정체성을 추천해주세요.
        아래 입력 정보를 바탕으로 이 사람에게 적합한 유튜브 채널 방향성과 정체성을 구체적으로 제안해주세요.
        ### 입력 정보:
        - 관심사 및 취미
        - 선호 콘텐츠 유형
        - 목표 시청자층
        - 영상 제작 가능 시간
        - 장비 및 예산
        - 콘텐츠 아이디어
        - 장기적인 목표
        
        ### 요청 사항:
        - 입력 정보를 종합적으로 분석하고, 각 항목에 초점을 맞추어 구체적인 제안을 작성하세요.
        - 각 정보가 서로 어떻게 연결되는지 고려하여 통합적인 방향성을 제시하세요.
        - 답변은 자연스러운 문장 서술 형태로 작성하되, 모든 입력 정보를 반영하세요.
        
        ### 참고:
        답변에는 다음 내용을 반드시 정확히 포함하세요:
        - 관심사와 선호 콘텐츠 유형, 목표 시청자층을 기반으로 한 채널 정체성 제안
        - 영상 제작 가능 시간을 고려한 업로드 일정 추천
        - 장비 및 예산 활용 방안 (제품 추천 포함)
        - 콘텐츠 아이디어의 구체적 활용법
        - 장기적인 목표 달성을 위한 조언"""
    },
    {
        "role": "user", 
        "content": f"""관심사 및 취미 : {answers["interest"]}
        선호 콘텐츠 유형 : {answers['contents']}
        목표 시청자층 : {answers['target']}
        영상 제작 가능 시간 : {answers['time']}
        장비 및 예산 : {answers['budget']}
        콘텐츠 아이디어 : {answers['creativity']}
        장기적인 목표 : {answers['goal']}
        
        이 정보를 바탕으로 적합한 유튜브 채널 방향성과 정체성을 제안해주세요.
        """
    }]

    completion_executor = CompletionExecutor(
        host=HYPERCLOVA_API_URL,
        api_key=HYPERCLOVA_API_KEY,
        request_id='309fa53d16a64d7c9c2d8f67f74ac70d'
    )
    request_data = {
                'messages': prompt,
                'topP': 0.8,
                'topK': 0,
                'maxTokens': 2000,
                'temperature': 0.35,
                'repeatPenalty': 5.0,
                'stopBefore': [],
                'includeAiFilters': False,
                'seed': 0
            }
    print(completion_executor.execute(request_data))

# 예제 실행 흐름

user_states = {'current_index':0, "answers":{}}
chatbot = Chatbot(user_states)
# 첫 번째 질문 시작
question, question_id = chatbot.get_next_question()
print("Chatbot:", question)

querys=["먹방", "리뷰, 먹방 등", "40대 이상 남자", "제한 없음", "일주일에 100만원", "상황극 하면서 먹기 (소개팅 상황극, 선후배 밥약속 상황극 등)", "실버 버튼"]
i=0
while question is not None:
    # 사용자 입력 시뮬레이션 (예: 실제로는 사용자의 입력값으로 대체)
    user_input = querys[i]#input("User: ")
    if not len(user_input):
        print('다시 입력해주세요')
        continue
    print(user_input)
    # 사용자 입력 처리 및 다음 질문 가져오기
    question, question_id = chatbot.handle_user_answer(question_id, user_input)
    
    if question:
        print("Chatbot:", question)
    else:
        print("Chatbot: 모든 질문이 완료되었습니다.")
    i += 1

# 모든 답변 취합 후 HyperCLOVA 호출
answers = user_states["answers"]
call_hyperclova(answers)


Chatbot: 어떤 분야에 관심이 많고, 주로 즐기는 취미는 무엇인가요? (예: 요리, 여행, 게임, 뷰티 등)
먹방
Chatbot: 영상 콘텐츠를 제작할 때 선호하는 유형이 있나요? (예: 브이로그, 리뷰, 튜토리얼, 예능 등)
리뷰, 먹방 등
Chatbot: 주 타겟으로 삼고 싶은 연령대나 성별, 지역 등이 있나요?
40대 이상 남자
Chatbot: 일주일에 영상 제작에 할애할 수 있는 시간이 어느정도 인가요?
제한 없음
Chatbot: 현재 보유하고 있는 장비와 앞으로의 예산 계획은 어떻게 되나요?
일주일에 100만원
Chatbot: 평소에 떠올렸던 재미있거나 유익한 콘텐츠 아이디어가 있다면 공유해주세요.
상황극 하면서 먹기 (소개팅 상황극, 선후배 밥약속 상황극 등)
Chatbot: 유튜버로서 이루고 싶은 장기적인 목표나 꿈이 있나요?
실버 버튼
Chatbot: 모든 질문이 완료되었습니다.
답변을 생성하고 있습니다...
목표 시청자층이 40대 이상 남성인 점을 고려할 때, '중년 아재들의 유쾌한 먹방'이라는 채널명으로 중년 남성들이 공감할 수 있는 다양한 상황에서 음식을 먹는 모습을 보여주는 것을 채널 정체성으로 설정하는 것을 추천드립니다. 소개팅 상황극, 직장 회식 상황극 등 일상 속에서 일어날 수 있는 여러 가지 상황을 연출하면서 음식을 먹는 모습을 통해 시청자들의 공감을 이끌어내고, 대리만족을 느낄 수 있도록 할 수 있습니다. 또한 이러한 방식은 기존의 단순한 먹방과는 차별화된 재미를 제공할 수 있을 것입니다.

영상 제작 가능 시간이 제한이 없으므로 매일 1개씩 꾸준히 올리는 것을 추천 드립니다. 이렇게 하면 구독자들과의 소통도 원활하게 이루어질 수 있으며, 새로운 시청자들을 유입시키는 데에도 도움이 됩니다.

장비 및 예산으로는 고해상도 카메라와 마이크를 구매하시는 것을 추천드립니다. 이를 통해 보다 높은 퀄리티의 영상을 제작할 수 있으며, 시청자들의 만족도를 높일 수 있습니다. 또 매주 100만원의 예산 중 일부를 인기있는 먹방 유튜버와의 